In [ ]:
import requests
import yfinance as yf
import pandas as pd
import time

# Set up the API key for your LLM service
api_key = "ljoWc7l91mdURBRNTNPlZQlGpV5OMo"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}

# Initialize portfolio
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}

# Function to execute trades based on LLM recommendation
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['NVDA'] = portfolio['positions'].get('NVDA', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'NVDA' in portfolio['positions'] and portfolio['positions']['NVDA'] > 0:
            shares_to_sell = portfolio['positions']['NVDA']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['NVDA'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")

# Function to calculate portfolio value
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value

# Fetch historical stock data for NVDA using yfinance
stock_data = yf.download('NVDA', start='2023-01-01', end='2023-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()

# Loop over each day in the stock data
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Nvidia (NVDA), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
    # Call the LLM service
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        # Ensure the recommendation is valid
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        # Execute the trade
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        # Optionally, pause or wait for the next time interval
        time.sleep(0.1)  # Small delay to simulate time between trades
    else:
        print(f"Error {response.status_code}: {response.text}")
        break  # Exit the loop on error

# Final portfolio value calculation
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100

# Display final portfolio performance
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)

# AI Trading Agent for Mid-Frequency Trading on Nvidia Stock (NVDA)

This code is a mid-frequency trading bot that uses an LLM (large language model) service to generate daily trading recommendations (buy, hold, sell) for Nvidia stock (NVDA). The bot simulates trading over a given historical period and tracks the portfolio's performance.

### Code Explanation

```python
import requests
import yfinance as yf
import pandas as pd
import time
```

- **Modules**: `requests` for API calls, `yfinance` for fetching historical stock data, `pandas` for data manipulation, and `time` for introducing delays in trading.

---

### 1. Setting Up the API Configuration for LLM Service

```python
api_key = "your_actual_api_key_here"  # Replace with your actual API key

# LLM Service Configuration
llm_url = "http://localhost:8000/chat/completions"
llm_headers = {
    "Content-Type": "application/json",
    "api-key": api_key,
}
```

- **API Key**: Used to authenticate requests to the LLM service.
- **LLM Service Configuration**: Defines the URL and headers required to make a request to the LLM model, which will provide trading recommendations.

---

### 2. Initializing the Portfolio

```python
portfolio = {
    'cash': 10000.0,  # Starting with $10,000
    'positions': {},  # Stock positions
    'transaction_history': []  # Record of transactions
}
```

- **Portfolio Initialization**: Starts with `$10,000` in cash, no positions, and an empty transaction history.

---

### 3. Function to Execute Trades

```python
def execute_trade(recommendation, current_price, date):
    if recommendation == 'buy':
        # Calculate the number of shares to buy (e.g., invest 10% of cash)
        investment_amount = portfolio['cash'] * 0.1
        shares_to_buy = investment_amount / current_price
        portfolio['cash'] -= investment_amount
        portfolio['positions']['NVDA'] = portfolio['positions'].get('NVDA', 0) + shares_to_buy
        portfolio['transaction_history'].append({
            'date': date,
            'action': 'buy',
            'shares': shares_to_buy,
            'price': current_price,
            'total': investment_amount
        })
        print(f"Bought {shares_to_buy:.2f} shares at ${current_price:.2f} per share on {date}.")
    elif recommendation == 'sell':
        if 'NVDA' in portfolio['positions'] and portfolio['positions']['NVDA'] > 0:
            shares_to_sell = portfolio['positions']['NVDA']
            proceeds = shares_to_sell * current_price
            portfolio['cash'] += proceeds
            portfolio['positions']['NVDA'] = 0
            portfolio['transaction_history'].append({
                'date': date,
                'action': 'sell',
                'shares': shares_to_sell,
                'price': current_price,
                'total': proceeds
            })
            print(f"Sold {shares_to_sell:.2f} shares at ${current_price:.2f} per share on {date}.")
        else:
            print(f"No shares to sell on {date}.")
    else:
        print(f"Holding position on {date}.")
```

- **execute_trade**: Executes trades based on the LLM's recommendation:
    - **Buy**: Invests 10% of cash to buy shares at the current price.
    - **Sell**: Sells all NVDA shares if owned.
    - **Hold**: Does nothing if the recommendation is to hold.

---

### 4. Calculating the Portfolio Value

```python
def calculate_portfolio_value(current_price):
    total_value = portfolio['cash']
    for stock, shares in portfolio['positions'].items():
        total_value += shares * current_price
    return total_value
```

- **calculate_portfolio_value**: Computes the portfolio's total value by adding cash and the current market value of any stock positions.

---

### 5. Fetching Historical Data for NVDA

```python
stock_data = yf.download('NVDA', start='2023-01-01', end='2023-12-31')

# Ensure the data is sorted by date and reset index
stock_data = stock_data.sort_index().reset_index()
```

- **stock_data**: Downloads Nvidia stock data for 2023 using `yfinance`.
- **Data Preparation**: Sorts and resets the index, making the data easier to loop through by date.

---

### 6. Looping Over Each Day in the Stock Data

```python
for index, row in stock_data.iterrows():
    # Prepare the prompt with recent data up to the current day
    recent_data = stock_data.loc[max(0, index - 4):index].to_string(index=False)
    date_str = row['Date'].strftime('%Y-%m-%d')
    prompt = f"""
You are a financial analyst. Based on the following recent stock data for Nvidia (NVDA), provide a buy, hold, or sell recommendation. Respond with only one word: buy, hold, or sell.

Stock Data:
{recent_data}

Recommendation:
"""
```

- **Prompt Preparation**: Sends a 5-day window of recent stock data to the LLM for recommendation. 
- **Looping Through Each Day**: Loops over each row of historical data to fetch and analyze recent stock prices for decision-making.

---

### 7. Calling the LLM Service for Recommendation

```python
    data = {
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "model": "llama3.1-8b",
        "service_name": "cerebras",
        "temperature": 0.7,
        "max_tokens": 50,
        "top_p": 0.9,
    }
    response = requests.post(llm_url, json=data, headers=llm_headers)
```

- **LLM Request**: Sends the prompt to the LLM with the required parameters to receive a trading recommendation (buy, hold, sell).

---

### 8. Executing the Trade

```python
    if response.status_code == 200:
        result = response.json()
        recommendation = result["content"].strip().lower()
        if recommendation not in ['buy', 'hold', 'sell']:
            print(f"Invalid recommendation '{recommendation}' on {date_str}. Skipping trade.")
            continue
        print(f"Date: {date_str}, LLM Recommendation: {recommendation}")
        current_price = row['Close']
        execute_trade(recommendation, current_price, date_str)
        time.sleep(0.1)
    else:
        print(f"Error {response.status_code}: {response.text}")
        break
```

- **Executing the Trade**: If the recommendation is valid, it executes the corresponding trade, with a short delay to simulate real-time trading.

---

### 9. Final Portfolio Value Calculation

```python
final_price = stock_data.iloc[-1]['Close']
final_portfolio_value = calculate_portfolio_value(final_price)
total_return = ((final_portfolio_value - 10000) / 10000) * 100
```

- **Final Value Calculation**: Computes the total return based on the final portfolio value at the end of the period.

---

### 10. Displaying Final Portfolio Performance

```python
print("\nFinal Portfolio Performance:")
print(f"Final Portfolio Value: ${final_portfolio_value:.2f}")
print(f"Total Return: {total_return:.2f}%")
print(f"Cash Available: ${portfolio['cash']:.2f}")
print(f"Positions: {portfolio['positions']}")
print("\nTransaction History:")
for transaction in portfolio['transaction_history']:
    print(transaction)
```

- **Performance Display**: Outputs the final portfolio value, total return, remaining cash, stock positions, and transaction history.

---

### Summary

This code provides a simulated trading environment, utilizing an LLM to analyze recent stock trends and make trading decisions. It calculates the portfolio value and tracks the overall performance, making it suitable for exploring mid-frequency trading strategies using AI-driven recommendations.

In [1]:
import asyncio
from crawl4ai import AsyncWebCrawler, CacheMode

async def main():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.nbcnews.com/business]")
        # Soone will be change to result.markdown
        print(result.markdown_v2.raw_markdown) 

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [2]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from crawl4ai import AsyncWebCrawler, CacheMode

async def main():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.nbcnews.com/business")
        # Assuming result has markdown_v2.raw_markdown as output
        print(result.markdown_v2.raw_markdown) 

if __name__ == "__main__":
    asyncio.run(main())

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_connection.py:272> exception=NotImplementedError()>
Traceback (most recent call last):
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_connection.py", line 279, in run
    await self._transport.connect()
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
   

NotImplementedError: 

In [2]:
import asyncio
from crawl4ai import AsyncWebCrawler

# Apply event loop policy for Windows compatibility
import sys
if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

async def main():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.nbcnews.com/business")
        print(result.markdown_v2.raw_markdown)

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
import asyncio
from crawl4ai import AsyncWebCrawler
import nest_asyncio

# Apply event loop policy for nested loops
nest_asyncio.apply()

# Apply event loop policy for Windows compatibility
import sys
if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

async def main():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.nbcnews.com/business")
        print(result.markdown_v2.raw_markdown)

if __name__ == "__main__":
    asyncio.run(main())

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2' coro=<Connection.run() done, defined at c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_connection.py:272> exception=NotImplementedError()>
Traceback (most recent call last):
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_connection.py", line 279, in run
    await self._transport.connect()
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\karan\AppData\Local\Programs\Python\Python312\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
   

NotImplementedError: 